In [1]:
import tensorflow as tf
from tensorflow import keras
import official.nlp.modeling.layers as nlp_layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.metrics import confusion_matrix
from numpy.random import RandomState
from sklearn.model_selection import StratifiedKFold
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import accuracy_score

2024-06-23 17:01:13.626057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 17:01:13.626112: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 17:01:13.626131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 17:01:13.630947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Loading

In [2]:
# use respective file name for each data_set 
# For Example (modified_data_SNR5,modified_data_SNR7,modified_data_SNR10)
data_file = open("Database_old_n_SNR20.csv",'r')
data_list = data_file.readlines()
data_file.close()
[no_of_samples,] = np.shape(data_list)
print (no_of_samples)

3600


In [3]:
vec = []
lab = []
for i in range (no_of_samples):
    all_values = data_list[i].split(',')
    all_values1 = np.asfarray(all_values[1:])
    array_data = np.array(all_values1, dtype = 'f')
    array_data = np.abs(array_data)
    array_data = array_data/max(array_data)
    y = all_values[0]
    vec.append(array_data)
    lab.append(y)
    pass
vec = np.reshape(vec,(no_of_samples,2048)) 
lab = np.reshape(lab,(no_of_samples,))

In [4]:
y_data = np.array(lab.astype(int)) #Labels are stored in y_data

In [5]:
array_data = vec
array_data = np.array(array_data, dtype = 'f') #Signal vectors are stored in array_data

# For Random Shuffling of Data

In [6]:
ran = np.arange(no_of_samples)
print (ran)
np.random.seed(seed = 0)
np.random.shuffle(ran)
print (ran)

[   0    1    2 ... 3597 3598 3599]
[1303 1009  764 ... 1653 2607 2732]


In [7]:
array_data = array_data[ran,:]
y_data = y_data[ran]

In [8]:
print (np.shape(array_data))
print (np.shape(y_data))

(3600, 2048)
(3600,)


# Splitting of Data into Training and Testing Datasets

In [9]:
skf = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [10]:
X_Train = []
X_Test = []
Y_Train = []
Y_Test = []
for train_index, test_index in skf.split(array_data,y_data):
    X_train, X_test = array_data[train_index], array_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    X_Train.append(X_train)
    X_Test.append(X_test)
    Y_Train.append(y_train)
    Y_Test.append(y_test)

# SNGP CNN Model

In [11]:
class UT_SNGP(tf.keras.Model):
    def __init__(self, num_classes, spec_norm_bound=0.9, conv_actfn = 'elu', actfn = 'relu', **kwargs):
        
        super().__init__()
        self.actfn = actfn
        self.conv_actfn = conv_actfn
        self.spec_norm_bound = spec_norm_bound
        self.num_classes = num_classes
        self.kwargs = kwargs

        # hidden layers.
        
        self.conv_layers1 = nlp_layers.SpectralNormalization(self.make_conv_layer(32, (1,25), (1,8)), norm_multiplier=self.spec_norm_bound)

        self.conv_layers2 = nlp_layers.SpectralNormalization(self.make_conv_layer(64, (1,3), (1,2)), norm_multiplier=self.spec_norm_bound)

        self.pool_layers1 = self.make_pool_layer((1,2),(1,2))

        self.conv_layers3 = nlp_layers.SpectralNormalization(self.make_conv_layer(64, (1,3), (1,2)), norm_multiplier=self.spec_norm_bound)

        self.pool_layers2 = self.make_pool_layer((1,2),(1,2))

        self.conv_layers4 = nlp_layers.SpectralNormalization(self.make_conv_layer(64, (1,3), (1,2)), norm_multiplier=self.spec_norm_bound)

        self.pool_layers3 = self.make_pool_layer((1,2),(1,2))

        self.flatten_layer = self.make_flat_layer()
        
        self.dense_layers1 = nlp_layers.SpectralNormalization(self.make_dense_layer(256), norm_multiplier=self.spec_norm_bound)
        self.dense_layers2 = nlp_layers.SpectralNormalization(self.make_dense_layer(256), norm_multiplier=self.spec_norm_bound)

        self.classifier = self.make_output_layer(num_classes)

    def call(self, inputs, training=True, return_covmat=False):
        
        conv1 = self.conv_layers1(inputs)
        conv2 = self.conv_layers2(conv1)
        pool1 = self.pool_layers1(conv2)
        conv3 = self.conv_layers3(pool1)
        pool2 = self.pool_layers2(conv3)
        conv4 = self.conv_layers4(pool2)
        pool3 = self.pool_layers3(conv4)
        flat = self.flatten_layer(pool3)
        dense1 = self.dense_layers1(flat)
        dense2 = self.dense_layers2(dense1)

        logits, covmat = self.classifier(dense2)

        if not training and return_covmat:
          return logits, covmat

        return logits

    def make_dense_layer(self, hidden_units):
        """Use the Dense layer as the hidden layer."""
        return tf.keras.layers.Dense(hidden_units, activation=self.actfn)

    def make_conv_layer(self, conv_filters, conv_kernel, conv_stride):
        """Use the Conv layer as the hidden layer."""
        return tf.keras.layers.Conv2D(filters = conv_filters, kernel_size= conv_kernel, strides = conv_stride, activation= self.conv_actfn,
                                      padding="SAME")

    def make_pool_layer(self, cpool_size, pool_stride):
        """Use the pool layer as the hidden layer."""
        return tf.keras.layers.MaxPooling2D(pool_size= cpool_size, strides = pool_stride, padding = "VALID")

    def make_flat_layer(self):
        """Use the flatten layer as the hidden layer."""
        return tf.keras.layers.Flatten()

    def make_output_layer(self, num_classes):
        """the output layer."""
        return nlp_layers.RandomFeatureGaussianProcess(num_classes,gp_cov_momentum=-1,
                                                       gp_kernel_scale_trainable=True, 
                                                       num_inducing=64, 
                                                       gp_kernel_scale=0.0001, **self.kwargs)

class ResetCovarianceCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        """Resets covariance matrix at the beginning of the epoch."""
        if epoch > 0:
            self.model.classifier.reset_covariance_matrix()

class UT_SNGPWithCovReset(UT_SNGP):
    def fit(self, *args, **kwargs):
        """Adds ResetCovarianceCallback to model callbacks."""
        kwargs["callbacks"] = list(kwargs.get("callbacks", []))
        kwargs["callbacks"].append(ResetCovarianceCallback())
        return super().fit(*args, **kwargs)

In [12]:
UT_model = UT_SNGP(num_classes = 5, spec_norm_bound=0.9, conv_actfn = 'elu', actfn = 'relu')

2024-06-23 17:01:27.900912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:01:27.907966: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:01:27.908001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:01:27.909681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:01:27.909726: I tensorflow/compile

In [13]:
UT_model.build((None,1,2048,1))

In [14]:
UT_model.summary()

Model: "ut_sngp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spectral_normalization (Sp  multiple                  889       
 ectralNormalization)                                            
                                                                 
 spectral_normalization_1 (  multiple                  6368      
 SpectralNormalization)                                          
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
 spectral_normalization_2 (  multiple                  12608     
 SpectralNormalization)                                          
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0   

In [15]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy(),
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)

In [16]:
UT_model = UT_SNGPWithCovReset(num_classes = 5)

In [17]:
train_config = dict(loss=loss, metrics=metrics, optimizer=optimizer)

In [18]:
UT_model.compile(**train_config)

In [19]:
i = 0
x_train = np.reshape(X_Train[i],[-1,1,2048,1])
x_test = np.reshape(X_Test[i],[-1,1,2048,1])
y_train = Y_Train[i]
y_test = Y_Test[i]

In [20]:
fit_config = dict(batch_size=600, epochs=1000)

In [21]:
H = UT_model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose = False,**fit_config)

2024-06-23 17:01:36.845971: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-06-23 17:01:36.928425: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [22]:
accu = pd.DataFrame(H.history)

In [23]:
test_accu=[]
test_accu.append(max(accu['val_sparse_categorical_accuracy']))

In [24]:
test_accu

[0.9944444298744202]

## File Saving and Uncertainty Calculations

In [25]:
def compute_posterior_mean_probability(logits, covmat, lambda_param=np.pi / 8.):
    # Computes uncertainty-adjusted logits using the built-in method.
    logits_adjusted = nlp_layers.gaussian_process.mean_field_logits(
    logits, covmat, mean_field_factor=lambda_param)
  
    #return tf.nn.softmax(logits_adjusted, axis=-1)[:, 0]
    return tf.nn.softmax(logits_adjusted, axis=-1)

In [26]:
UT_logits, UT_covmat = UT_model(x_test, training = False, return_covmat=True)

2024-06-23 17:03:40.154025: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x1fe9f960


In [27]:
UT_probs = compute_posterior_mean_probability(UT_logits, UT_covmat)

In [28]:
UT_probs

<tf.Tensor: shape=(360, 5), dtype=float32, numpy=
array([[0.98463535, 0.0051801 , 0.00196744, 0.00418706, 0.00403009],
       [0.97684485, 0.00603159, 0.00266852, 0.00427389, 0.01018109],
       [0.00339106, 0.9921531 , 0.0014965 , 0.00132811, 0.00163125],
       ...,
       [0.00528874, 0.9896107 , 0.00165717, 0.00154405, 0.00189935],
       [0.00500088, 0.98964816, 0.00188019, 0.00168828, 0.00178252],
       [0.00503237, 0.9874686 , 0.00252173, 0.00262658, 0.00235076]],
      dtype=float32)>

In [29]:
uncertainty = UT_probs * (1. - UT_probs)

In [30]:
df_uncertain = pd.DataFrame(uncertainty,columns = ['C','LF','LP','P','S'])

In [31]:
df_uncertain.to_csv(r'SNGP_UT_Uncertainty_SNR20.csv')

In [32]:
df_lab = pd.DataFrame(y_test,columns = ['labels'])

In [33]:
df_lab.to_csv(r'SNGP_UT_Labels_SNR20.csv')

In [34]:
df_prob = pd.DataFrame(UT_probs,columns = ['C','LF','LP','P','S'])

In [35]:
df_prob.to_csv(r'SNGP_UT_Probs_SNR20.csv')

In [36]:
for i in range(5):
    na = i
    a = df_lab.labels[df_lab.labels==na].index.tolist()
    col = ['C','LF','LP','P','S']
    col_l = col[na]
    b = df_uncertain[col_l]
    c = b[a]
    avg = np.average(c)
    print(col[na])
    print(avg)

C
0.023152513
LF
0.03952577
LP
0.03961556
P
0.022898976
S
0.03452117
